# Reverse Engineering the UR3e IK parameters

Universal Robots posts the [DH parameters](https://www.universal-robots.com/articles/ur/application-installation/dh-parameters-for-calculations-of-kinematics-and-dynamics/) of all of their robots online, the hope is with these parameters for our robot, and the joint position read off the modbus, we might be able to reverse-engineering the inverse kinematics and write our own controller for working with a floating coordinate system.

In [24]:
import numpy as np
import json
import os
import time

In [41]:
DH = np.array([[0, 0, 0.1519,np.pi/2],
                    [0,-0.2434,0.,0.],
                    [0,-0.2132,0.,0.],
                    [0,0.,0.1311,np.pi/2],
                    [0,0.,0.0854,-np.pi/2],
                    [0,0.,0.0921,0.]])

print("Available files:")
files = []
for file in os.listdir():
    if file.endswith(".json"):
        print("\t" + file)
        files.append(file)

i = 7

path = os.path.abspath("..\\IK_Scans") + "\\" + files[i]

print(path)

with open(path, 'r') as infile:
    json_data = json.load(infile)


start_joints = json_data["Start joints"]
joints_at_points = json_data["joints_at_points"]

print(start_joints)

print(joints_at_points[0][2])

Available files:
	Test_08_07__17_50.json
	Test_08_07__18_03.json
	Test_08_07__18_09.json
	Test_08_07__18_16.json
	Test_08_07__18_22.json
	Test_08_07__18_26.json
	Test_08_11__14_51.json
	Test_08_11__15_02.json
	Test_08_11__15_06.json
c:\Users\ander\Documents\COBOT-Transducer-Control-Code\IK_Scans\Test_08_11__15_02.json
[[3.196], [5.368], [1.623], [3.658], [5.913], [1.874]]
[[[0.4015], [0.4006], [0.1435]], [[0.009], [2.513], [1.748]]]


In [42]:
#               theta,a,d,alpha     (radians and meters)
DH = np.array([[0, 0, 0.1519,np.pi/2],
                    [0,-0.2434,0.,0.],
                    [0,-0.2132,0.,0.],
                    [0,0.,0.1311,np.pi/2],
                    [0,0.,0.0854,-np.pi/2],
                    [0,0.,0.0921,0.]])


start_joints = np.array(start_joints).flatten()

DH[:,0] = start_joints

print(DH)

# print(DH[0,:])

joints=[]

#=-(SIN(O6))*COS(R6)

# O = DH[:,0], P = DH[:,1], Q = DH[:,2], R = DH[:,3]

for i in range(6):
    joints.append(np.array([[np.cos(DH[i,0]), -np.sin(DH[i,0]) * np.cos(DH[i,3]), np.sin(DH[i,0]) * np.sin(DH[i,3]), DH[i,1] * np.cos(DH[i,0])],
                            [np.sin(DH[i,0]), np.cos(DH[i,0]) * np.cos(DH[i,3]), -np.cos(DH[i,0]) * np.sin(DH[i,3]), DH[i,1] * np.sin(DH[i,0])],
                            [0., np.sin(DH[i,3]), np.cos(DH[i,3]),DH[i,2]],
                            [0.,0.,0.,1.]]))

(T1,T2,T3,T4,T5,T6) = tuple(joints)
# print(T1)
t = T1
for i in range(5):
    print(i)
    print(t)
    t = np.matmul(t,joints[i+1])

print(t)

final_result = np.matmul(np.matmul(np.matmul(np.matmul(np.matmul(T1,T2),T3),T4),T5),T6)

print("-----------------")
print(final_result)

def matrix_to_coordinates(mat):
    X = mat[0,3]
    Y = mat[1,3]
    Z = mat[2,3]

    beta = np.arctan2(-mat[2,0], (mat[0,0]**2 + mat[1,0]**2)**0.5)
    alpha = np.arctan2(mat[1,0]/np.cos(beta), mat[0,0]/np.cos(beta))
    gamma = np.arctan2(mat[2,1]/np.cos(beta), mat[2,2]/np.cos(beta))
    return X,Y,Z,beta,alpha,gamma

print(matrix_to_coordinates(final_result))

[[ 3.196       0.          0.1519      1.57079633]
 [ 5.368      -0.2434      0.          0.        ]
 [ 1.623      -0.2132      0.          0.        ]
 [ 3.658       0.          0.1311      1.57079633]
 [ 5.913       0.          0.0854     -1.57079633]
 [ 1.874       0.          0.0921      0.        ]]
0
[[-9.98520285e-01  3.32984575e-18 -5.43805080e-02 -0.00000000e+00]
 [-5.43805080e-02 -6.11417336e-17  9.98520285e-01 -0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  6.12323400e-17  1.51900000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
1
[[-6.08741599e-01 -7.91502638e-01 -5.43805080e-02  1.48167705e-01]
 [-3.31527340e-02 -4.31061003e-02  9.98520285e-01  8.06937544e-03]
 [-7.92675572e-01  6.09643697e-01  6.12323400e-17  3.44837234e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
2
[[-7.58660257e-01  6.49212889e-01 -5.43805080e-02  3.09914072e-01]
 [-4.13174682e-02  3.53568447e-02  9.98520285e-01  1.68782597e-02]
 [ 6.50174962e-

In [34]:
print(T6)

[[ 0.24526819 -0.96945527  0.          0.        ]
 [ 0.96945527  0.24526819 -0.          0.        ]
 [ 0.          0.          1.          0.0921    ]
 [ 0.          0.          0.          1.        ]]
